In [1]:
from clamv import *
import cv2

In [2]:
X, y = get_images()

In [ ]:
test(OpenCVClassifier(cv2.face.createEigenFaceRecognizer), X, y)

(0.58656924449916814, 0.056801596825888756)

In [3]:
test(EigenClassifier(), X, y)

(0.57698536797657274, 0.040295840076542376)

In [4]:
test(OpenCVClassifier(cv2.face.createFisherFaceRecognizer), X, y)

(0.71804585236085305, 0.039591290965637224)

In [4]:
pca_params = {
    'n_components': 150,
    'svd_solver': 'randomized',
    'whiten': True,
}
svc_params = {
    'kernel': 'rbf',
    'class_weight': 'balanced',
}
param_grid = {
    'C': [1e3, 5e3, 1e4, 5e4, 1e5],
    'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
}
test(EigenClassifierGridSearch(pca_params, svc_params, param_grid), X, y)

(0.85479823228778895, 0.035829553692219233)